In [6]:
import cv2 as cv
import numpy as np

In [7]:
cv.destroyAllWindows()

In [8]:
def image_transform(image: np.ndarray):
    # remove gaussian noise
    image = cv.GaussianBlur(image, (3, 3), 0)

    # quantize image
    N = 50
    image = np.round(image * (N / 255), decimals=0) * (255 / N)

    return image.astype(np.uint8)

In [9]:
from sample_extractor import SampleExtractor
from view_controller import VideoStream

vid = VideoStream("single_worm1.avi", color_conversion=cv.COLOR_BGR2GRAY)

extractor = SampleExtractor(vid, image_transform)

extractor.calc_video_bboxes()
extractor.generate_samples(5, 200, 200, "samples2/sample{}.avi")

calculating background:   9%|▉         | 9/100 [00:00<00:01, 84.59it/s]

creating samples: 100%|██████████| 5/5 [00:00<00:00, 13.41it/s]


In [5]:
import time

video = VideoStream("samples2/sample3.avi")

for i, frame in enumerate(video):
    cv.imshow("vid", frame)
    #time.sleep(0.1)
    if cv.waitKey(10) & 0xFF == ord("q"):
        break

cv.destroyAllWindows()

AssertionError: 

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes

In [ ]:
from view_controller import VideoStream
from box_extractor import BoxExtractor

video = VideoStream("worms.avi", padding_size=(251, 251))
extractor = BoxExtractor(find_boxes, slice_size=251, object_size=150)

for frame in video:
    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    head_coords, rois = extractor.run(frame)

    # Draw bboxes
    for box in head_coords:
        x, y, w, h = box
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)

    cv.imshow("BBOXES", frame)

In [ ]:
import math
from view_controller import ViewController

sim = ViewController("worms.avi", color_conversion=cv.COLOR_BGR2GRAY)

fps = 40
micro_view_time = 200
tracking_time = 50
moving_time = 25

frame_time = 1 / fps
micro_view_frames = math.ceil(micro_view_time / frame_time)
tracking_frames = math.ceil(tracking_time / frame_time)
moving_frames = math.ceil(moving_time / frame_time)

while True:
    # Here we assume that the worm is kinda centered on the screen
    sim.micro_view()  # Takes 'micro_frames' frames, can do other stuff meanwhile
    sim.next_frame(micro_view_frames - tracking_frames)

    cam_view = sim.camera_view()
    dx, dy = real_time_tracking(cam_view)  # Takes 'track_frames' frames, can't do anything meanwhile
    sim.next_frame(tracking_frames)  # micro and tracking finished

    sim.next_frame(moving_frames)  # moving takes 'move_frames' frames, can do other stuff meanwhile
    sim.move_position(dx, dy)